# Set Up (imports including psspy)

In [42]:
!pip install pandas
!pip install numpy

In [43]:
import pandas as pd
import numpy as np
import os, sys
print(sys.version) # should be 3.11.9

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [2]:
print(sys.path)

['C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv', '', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\Pythonwin']


In [3]:
PSSE_LOCATION = r"C:\Program Files\PTI\PSSE36\36.1\PSSPY311" # this is where the psspy module is located on my machine
sys.path.append(PSSE_LOCATION)
os.environ['PATH'] += ';' + PSSE_LOCATION

In [4]:
import psse3601
import psspy

# Import Case Data

In [44]:
# initialize system
psspy.psseinit(39)

0

In [45]:
# read case
psspy.read(0, "IEEE 39 bus.RAW")


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 39 records for Bus data
 Starting processing of Load data records
 Finished processing of 31 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 2 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 10 records for Generator data
 Starting processing of Branch data records
 Finished processing of 34 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 48 records for 12 Transformer data
 Starting processing of Area data records
 Finished processing of 1 records for Area data
 Starting processing of 

0

In [46]:
# run load flow analysis
psspy.fnsl([0,0,0,1,1,0,99,0]) # flat start newton-raphson load flow


 Ordering network...
 Diagonals = 38   Off-diagonals = 66   Maximum size = 90

  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0         0.0007(      2     )    0.0022(      5     )
                                                             0.00001(     17     )   0.00000(     22     )
   1         0.0000(      2     )    0.0000(      5     )
                                                             0.00000(     18     )   0.00000(     25     )
   2         0.0000(      6     )    0.0000(     22     )


 Reached tolerance in 2 iterations

 Largest mismatch:     -0.00 MW      0.00 Mvar      0.00 MVA at bus 5 [5           1.0000]
 System total absolute mismatch:                    0.00 MVA

 SWING BUS SUMMARY:
   BUS#-SCT X-- NAME --X BASKV      PGEN     PMAX    PMIN      QGEN     QMAX    QMIN
     31     31          1.0000     571.3   9999.9     0.0     363.9    800.0  -500.0


0

In [67]:
from itertools import groupby
from operator import itemgetter
attr_type = itemgetter(0)

def subsystem_info(name, attributes, sid=-1, inservice=True):
    """
    Returns requested attributes from the PSS(r)E subsystem API
    for the given subsystem id and subsystem element name.
    e.g. to retrieve bus attributes "NAME", "NUMBER" and "PU"
      
      subsystem_info('bus', ["NAME", "NUMBER", "PU"])
    where the 'bus' `name` argument comes from the original
    PSS(r)E subsystem API naming convention found in Chapter 8 of the
    PSS(r)E API.
    abusint  # bus
    amachint # mach
    aloadint # load
    Args:
      inservice [optional]: True (default) to list only information
         for in service elements;
      sid [optional]: list only information for elements in this
         subsystem id (-1, all elements by default).  
    
    """
    name = name.lower()
    gettypes = getattr(psspy, 'a%stypes' % name)
    apilookup = {
            'I': getattr(psspy, 'a%sint' % name),
            'R': getattr(psspy, 'a%sreal' % name),
            'X': getattr(psspy, 'a%scplx' % name),
            'C': getattr(psspy, 'a%schar' % name), }

    result = []
    ierr, attr_types = gettypes(attributes)

    df = pd.DataFrame(columns=attributes)

    df_col = 0
    for k, group in groupby(list(zip(attr_types, attributes)), key=attr_type):
        func = apilookup[k]
        strings = list(list(zip(*group))[1])
        ierr, res = func(sid, flag=1 if inservice else 2, string=strings)
        df[attributes[df_col]] = res[0]
        df_col += 1

    return df

In [68]:
# cols = ['NUMBER', 'PU', 'Angle']
cols = ['PU', 'NUMBER']
businfo = subsystem_info('bus', cols, sid=-1)
businfo.head()

,PU,NUMBER
0,1.051784,1
1,1.059875,2
2,1.056548,3
3,1.059186,4
4,1.073595,5


In [20]:
ierr, busIDs = psspy.abusint(sid=-1, string="ID")
ierr, busNums = psspy.abusint(sid=-1, flag=4, string="NUMBER")
print(busIDs)

None


In [ ]:
print(busNums)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 39]]


In [18]:
ierr, busNames = psspy.abuschar(sid=-1, string="NAME")
print(busNames)

[['1           ', '2           ', '3           ', '4           ', '5           ', '6           ', '7           ', '8           ', '9           ', '10          ', '11          ', '12          ', '13          ', '14          ', '15          ', '16          ', '17          ', '18          ', '19          ', '20          ', '21          ', '22          ', '23          ', '24          ', '25          ', '26          ', '27          ', '28          ', '29          ', '30          ', '31          ', '32          ', '33          ', '34          ', '35          ', '36          ', '37          ', '38          ', '39          ']]
